In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-22"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15407,2024-03-22,Brasil Nbb,20:00,Mogi,Cerrado,1.99,1.74,155.5,1.85,1.85,1.5,1.85,1.61,MooYq9Cj,0.502513,0.574713,0.540541,0.540541,0.077225,0.4,0.6,NaN,NaN,270.922,165.552399,0.611070,262.176,98.136926,0.374317,195.520,225.302,129.60,212.42,0.0,0.0,0.0,0.0,0.094786,0.000000,0.098096,-3.35,-0.670,-1.477612,0.418474,0.2,-0.218474,0.00,0.000,inf,0.297626,0.3,0.002374
15408,2024-03-22,Brasil Lbf Feminina,21:00,Sodie Mesquita F,Blumenau F,1.86,1.86,136.5,1.85,1.81,-1.5,2.01,2.01,Wxb5qT7B,0.537634,0.537634,0.540541,0.552486,0.075269,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.015456,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
15409,2024-03-22,Europa Euroliga,15:00,Crvena zvezda,Virtus Bologna,2.05,1.80,157.5,1.91,1.91,1.5,1.91,1.63,l4i1Xhxl,0.487805,0.555556,0.523560,0.523560,0.043360,0.4,0.6,NaN,NaN,198.692,103.579021,0.521304,180.746,21.143625,0.116980,146.578,194.706,191.52,192.78,0.0,0.0,0.0,0.0,0.091832,0.000000,0.111859,-3.88,-0.776,-1.353093,0.685499,0.5,-0.185499,-5.00,-1.000,-0.800000,0.441790,0.2,-0.241790
15410,2024-03-22,Europa Euroliga,15:00,Monaco,Valencia,1.19,4.91,160.5,1.91,1.91,-10.5,2.00,10.50,Sfj5WCif,0.840336,0.203666,0.523560,0.523560,0.044002,0.6,0.2,NaN,NaN,134.336,32.630297,0.242901,163.682,39.156518,0.239223,139.240,254.484,118.58,135.27,0.0,0.0,0.0,0.0,0.862438,0.000000,0.961665,2.71,0.542,0.350554,0.678516,0.8,0.121484,4.22,0.844,4.632701,0.000000,0.0,0.000000
15411,2024-03-22,Europa Euroliga,16:15,Panathinaikos,Barcelona,1.58,2.62,157.5,1.85,2.09,-4.5,2.01,3.75,AJuAVW60,0.632911,0.381679,0.540541,0.478469,0.014591,0.0,0.4,NaN,NaN,169.698,88.909023,0.523925,140.560,37.104781,0.263978,121.400,152.552,116.92,190.18,0.0,0.0,0.0,0.0,0.350186,0.086145,0.427210,2.44,0.488,1.188525,0.664445,0.9,0.235555,-1.47,-0.294,-5.510204,0.560546,0.4,-0.160546
15412,2024-03-22,Europa Euroliga,16:30,Bayern,Maccabi Tel Aviv,1.80,2.05,165.5,1.91,1.91,-2.5,2.00,2.30,OMqEUjM6,0.555556,0.487805,0.523560,0.523560,0.043360,0.4,0.8,NaN,NaN,143.792,52.511693,0.365192,133.066,31.870224,0.239507,113.232,170.190,103.04,172.20,0.0,0.0,0.0,0.0,0.091832,0.000000,0.098666,0.51,0.102,7.843137,0.707595,0.7,-0.007595,-0.08,-0.016,-65.625000,0.549046,0.5,-0.049046
15413,2024-03-22,Europa Euroliga,16:30,Milano,Fenerbahce,2.05,1.80,159.5,1.91,1.91,1.5,1.91,1.63,IqqITAyD,0.487805,0.555556,0.523560,0.523560,0.043360,0.4,0.4,NaN,NaN,172.314,42.927145,0.249122,131.528,27.869462,0.211890,144.716,143.846,151.30,101.76,0.0,0.0,0.0,0.0,0.091832,0.000000,0.111859,2.76,0.552,1.902174,0.607852,0.7,0.092148,-1.25,-0.250,-3.200000,0.724144,0.7,-0.024144
15414,2024-03-22,Alemanha Bbl,16:00,Rostock,Bamberg,1.74,2.10,182.5,1.81,1.95,-2.5,1.99,2.40,WMguOiwg,0.574713,0.476190,0.552486,0.512821,0.050903,0.6,0.4,NaN,NaN,243.708,65.623405,0.269271,173.416,37.459873,0.216012,225.552,278.636,223.30,202.80,0.0,0.0,0.0,0.0,0.132583,0.052657,0.132079,-2.69,-0.538,-1.375465,0.000000,0.0,0.000000,-5.00,-1.000,-1.100000,0.000000,0.0,0.000000
15415,2024-03-22,Arábia Saudita Premier League,16:00,Al Taawon,Dbae,1.88,1.79,170.5,1.83,1.83,-1.5,2.05,1.91,vJ51L83F,0.531915,0.558659,0.546448,0.546448,0.090574,0.4,0.0,NaN,NaN,1063.770,378.413348,0.355729,625.268,220.604967,0.352817,642.770,0.000,787.50,735.00,1.0,1.0,0.0,0.0,0.034681,0.000000,0.049997,-5.00,-1.000,-0.880000,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Brasil Lbf Feminina,Sodie Mesquita F,Blumenau F,136.5,1.85,1.0000,Over
1,15:00,Europa Euroliga,Monaco,Valencia,160.5,1.91,1.0000,Over
2,16:15,Europa Euroliga,Panathinaikos,Barcelona,157.5,1.85,1.0000,Over
3,16:30,Europa Euroliga,Bayern,Maccabi Tel Aviv,165.5,1.91,1.0000,Over
4,16:00,Alemanha Bbl,Rostock,Bamberg,182.5,1.81,1.0000,Over
5,16:00,Arábia Saudita Premier League,Al Taawon,Dbae,170.5,1.83,0.9986,Over
6,05:30,Austrália Nbl,Tasmania JackJumpers,Melbourne United,179.5,1.89,1.0000,Over
7,15:00,Áustria Superliga,UBSC Graz,Traiskirchen,152.5,1.87,1.0000,Over
8,20:30,Chile Lnb,Puente Alto,Ancud,160.5,1.87,1.0000,Over
9,08:35,China Cba,Beijing Royal Fighters,Beijing,200.5,1.82,0.9894,Over
